# Orca Photonic Device Tutorial
This notebook will guide you through the basics of understanding and playing with photonic circuits using the Orca device.

In [ ]:
# First, perform the relevant imports and navigate to the root folder
import os
import numpy as np
import matplotlib.pyplot as plt

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

from ptseries.tbi import create_tbi

***ptseries*** is a library created by Orca Computing specifically to work with the Orca photonic divices.

## 1. Define a Photonic Circuit
A photonic circuit allows for the manipulation of photons at a microscopic scale for various computational and experimental applications.

In [ ]:
# Creating a Photonic circuit
tbi = create_tbi()

### Task 1: Change input state and draw the circuit
Define your own input state and draw the circuit using the following method:

In [ ]:
input_state = ...
tbi.draw(input_state=input_state, padding=1)

### Task 2: Run the circuit
In the previous cell you already created the circuit with some number of loops so now we want to run the circuit. To do that we need to set theta angles (in radians). The number of thetas is dependent on the number of loops and input size. It can be calculated as:

*<h5><center> (input_size - 1) * n_loops </center></h5>*

Also you have to set up how many times you want to run the circuit (parameter called "n_samples")


In [ ]:
# Set up the parameters
theta_list = ...
n_samples = ...

# Draw samples
samples = tbi.sample(
    input_state=input_state,
    theta_list=theta_list,
    n_samples=n_samples
    )

# Plot a histogram of the counts
samples_sorted = dict(sorted(samples.items(), key=lambda state: state[0]))
labels = list(samples_sorted.keys())
labels = [str(i) for i in labels]
values = list(samples_sorted.values())
plt.bar(range(len(values)), values, tick_label=labels)
plt.xticks(rotation=90)
plt.show()

The plot represents how many times we got each of states presented on the x-axis. As you can notice, the sum of values in each state is equal to the sum of ones in the input state but it is impossible to get an output state with all ones.

That is happening because of entanglement of photons inside the circuit. 

You can change the input_state, theta_list and n_samples to check correctness of this property yourself. This should help to understand the logic of the divice better.

### Task 3: Change number of loops
In the Orca photonic circuits there is a parameter called "n_loops". On the drawing you created above you can see the layer of thetas. Those are repesenting angles for rotation of photons. One layer of them called "a loop". In function "create_tbi" you have a chance to set up the number of loops. Try to do that and draw the circuit to see how it looks:

In [ ]:
tbi = create_tbi()
tbi.draw(input_state=input_state, padding=1)

Now we can run the new circuit. Remember that number of thetas has to be greater. You can use the formula above to calculate it. Also because of increasing number of thetas the circuit will take longer to run.

In [ ]:
# Set up the parameters
theta_list = ...
n_samples = ...

# Draw samples
samples = tbi.sample(
    input_state=input_state,
    theta_list=theta_list,
    n_samples=n_samples
    )

# Plot a histogram of the counts
samples_sorted = dict(sorted(samples.items(), key=lambda state: state[0]))
labels = list(samples_sorted.keys())
labels = [str(i) for i in labels]
values = list(samples_sorted.values())
plt.bar(range(len(values)), values, tick_label=labels)
plt.xticks(rotation=90)
plt.show()

#### Congrats! Now you know the basics! So we can move to harder tasks such as solving MaxCut problem using Binary Bosonic Solver.

## 2. Solving Portfolio Optimization Problem with Binary Bosonic Solver
This part of the tutorial will guide you through understanding of the algorithm for solving optimization problems and show the principal of work using an instance of one the famous optimization problems.

The Portfolio Optimization problem is a fundamental task in financial management. It involves selecting the proportion of various assets in a portfolio to maximize returns while minimizing risk, typically measured as the variance or volatility of returns. The challenge is to find the optimal balance between expected return and risk, adhering to constraints such as budget limits and risk tolerance. This problem is NP-hard, making it computationally challenging to solve exactly, especially as the number of assets increases.

Let's start by generating a random set of assets with given expected returns and covariances. Then, we will use the Binary Bosonic Solver to find the maximum cut.

In [ ]:
# First we will import all necessary libraries and functions
import itertools 
import random

from ptseries.algorithms.binary_solvers.binary_bosonic_solver import BinaryBosonicSolver
from ptseries.common.logger import Logger

Then we can generate random instances.

In [ ]:
# Function for covariance matrix
def generate_covariance_matrix(n_assets):
    random_matrix = np.random.randn(n_assets,n_assets)
    cov_matrix = np.dot(random_matrix,random_matrix.T) # semi definite matrix
    return cov_matrix

# Function for expected returns
def generate_expected_returns(n_assets):
    return np.random.randn(n_assets)

Or we can use real data instead.

In [ ]:
# Define the stock price data as a NumPy array
stock_prices = np.array([
    [93.043,  51.826, 1.063],
    [84.585,  52.823, 0.938],
    [111.453, 56.477, 1.000],
    [99.525,  49.805, 0.938],
    [95.819,  50.287, 1.438],
    [114.708, 51.521, 1.700],
    [111.515, 51.531, 2.540],
    [113.211, 48.664, 2.390],
    [104.942, 55.744, 2.130],
    [99.827,  47.916, 2.980],
    [91.607,  49.438, 1.900],
    [107.937, 51.336, 1.750],
    [115.590, 55.081, 1.800]
])

# Columns correspond to: IBM, WMT, SEHI
# Rows correspond to months from November-00 to November-01

In [ ]:
# Define the stock returns data as a NumPy array
stock_returns = np.array([
    [-0.091,  0.019, -0.118],
    [0.318,  0.069,  0.067],
    [-0.107, -0.118, -0.063],
    [-0.037,  0.010,  0.500],
    [0.197,  0.025,  0.183],
    [-0.028, -0.002,  0.849],
    [0.015, -0.056,  0.309],
    [0.003,  0.130, -0.107],
    [-0.049, -0.142,  0.045],
    [-0.082,  0.042, -0.362],
    [0.178,  0.038, -0.079],
    [0.071,  0.073,  0.029]
])

# Average returns
average_returns = np.array([0.026, 0.008, 0.074])

In [ ]:
# Define the covariance matrix as a NumPy array
covariance_matrix = np.array([
    [0.017087987, 0.003298885, 0.001224849],
    [0.003298885, 0.005909044, 0.004488271],
    [0.001224849, 0.004488271, 0.063000818]
])


Let's make the visual representation of this dataset

In [ ]:
# Assuming equal weights for simplicity
weights = np.array([1/3, 1/3, 1/3])

# Calculate portfolio return and volatility (standard deviation)
portfolio_return = np.dot(stock_returns.mean(axis=0), weights)
portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

# Simulate multiple portfolios with random weights
n_portfolios = 1000
results = np.zeros((n_portfolios, 3))

for i in range(n_portfolios):
    weights = np.random.random(3)
    weights /= np.sum(weights)
    
    portfolio_return = np.dot(stock_returns.mean(axis=0), weights)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    
    sharpe_ratio = portfolio_return / portfolio_volatility
    
    results[i] = [portfolio_volatility, portfolio_return, sharpe_ratio]

# Plotting
plt.figure(figsize=(10, 5))
plt.scatter(results[:, 0], results[:, 1], c=results[:, 2], cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility', fontsize=12, fontweight='bold')
plt.ylabel('Return', fontsize=12, fontweight='bold')
plt.title('Efficient Frontier', fontsize=14, fontweight='bold')
plt.grid(True)
plt.show()

Now when we have an instance we will solve, which means find the most optimal investments with the smallest risks. We will use a binary bosonic solver. A binary bosonic solver (BBS) is a computational tool used to solve optimization problems by modeling them with bosonic particles, which are particles that follow Bose-Einstein statistics. In this context, the problem is represented in a binary form, where variables take on values of 0 or 1. The bosonic solver leverages quantum mechanical principles, particularly the behavior of bosons, to explore the solution space. By simulating the interactions and energy states of these particles, the solver aims to find the optimal configuration that minimizes or maximizes a given cost function. This approach is especially relevant in quantum computing and advanced optimization algorithms.

In [ ]:
# Function with application of the BBS
def BBS(state, qubo_function, tbi_params={"tbi_type": "single-loop"}, gradient_mode = "spsa"):
    problem_size = len(state)
    logger = Logger(log_dir=None)
    
    bbs = BinaryBosonicSolver(
        problem_size,
        qubo_function,
        tbi_params=tbi_params,
        gradient_mode=gradient_mode
    )

    bbs.train(
        learning_rate=1e-2,
        updates=160,
        print_frequency=20,
        logger=logger
    )

    best_sol, best_energy = bbs.return_solution()

    return best_sol, best_energy

To use the BBS we will have to create a cost function. To do that we will transform the problem instance into a QUBO matrix. A QUBO (Quadratic Unconstrained Binary Optimization) matrix is a mathematical representation used in optimization problems where the goal is to minimize or maximize a quadratic objective function of binary variables. The matrix encodes the coefficients of the quadratic and linear terms, making it suitable for solving problems using methods like quantum annealing and classical optimization algorithms.

In [ ]:
# Function for creating a QUBO matrix
def portoflio_qubo_matrix(num_assets, covariance_matrix=None, expected_returns=None, risk=0.5):
    if expected_returns is None:
        expected_returns = np.random.uniform(low=1, high=10, size=num_assets)
    
    if covariance_matrix is None:
        random_matrix = np.random.uniform(low=0, high=1, size=(num_assets, num_assets))
        covariance_matrix = np.dot(random_matrix, random_matrix.T)

    Q = np.zeros((num_assets, num_assets))
    for i in range(num_assets):
        Q[i, i] = -expected_returns[i] + risk * covariance_matrix[i, i]
        for j in range(i + 1, num_assets):
            Q[i, j] = 2 * risk * covariance_matrix[i, j]
            Q[j, i] = Q[i, j]
    return Q

### Task 1: Create a Cost Function
The next step is to create a cost function based on the QUBO matrix. The cost function evaluates the "cost" or "energy" associated with a binary vector in the context of the optimization problem. The QUBO matrix \( Q \) defines the relationship between the variables, and the cost function computes the value by applying this matrix to the binary vector.

To achieve this, the cost function should accept a binary vector as input and calculate its cost using the quadratic form:

Cost = bin_vec^T * Q * bin_vec

In Python, this can be implemented using the `np.dot` function, which performs the necessary matrix multiplications.

In [ ]:
Q = ...

def cost_function(bin_vec):
    return ...

### Task 2: Apply BBS to the Instance
The last step is to solve the problem. To do that just get the solution and energy from BBS. Also you can play around with tbi_parameters, gradient_mode and other parameters inside BBS function.

In [ ]:
initial_state = ...
solution, energy = BBS(...)
print(solution, energy)

#### Congrats! Now you know how Binary Bosonic Solver works! So we can move to harder tasks such as hybrid neural networks.

## 3. Creating a Hybrid Neural Network
This part of the tutorial will guide you through understanding both classical and quantum neural networks and how they can be integrated to form hybrid systems.

#### What are Neural Networks?
Neural networks are computational models that mimic the human brain's structure and function, using layers of nodes or neurons to process data.

#### How do Neural Networks Work?
A neural network processes input data through layers to make predictions. It learns by adjusting its neurons' weights based on the errors in its predictions.

A feedforward neural network is one of the simplest types of artificial neural networks devised. In this network, the information moves in only one direction—forward—from the input nodes, through the hidden nodes (if any), and to the output nodes. There are no cycles or loops in the network. Feedforward neural networks were the first type of artificial neural network invented and are simpler than their counterparts like recurrent neural networks and convolutional neural networks. You can read more here: https://deepai.org/machine-learning-glossary-and-terms/feed-forward-neural-network

Any neural network contains the following parts:
1. Input Layer
2. Hidden Layers
3. Output Layer
4. Activation Function
5. Loss Function
6. Optimizer

Firstly we need to understand what is the layer of neural network. There are several types or configurations of layers but easiest of them is linear layers so we will take a look at its structure. You can about other types of layers here: https://pytorch.org/docs/stable/nn.html#convolution-layers

Each linear layer consists of n number of neurons. For example number of neurons in input layer is equal to the number of inputs. In MNIST dataset (famous image dataset) size of images is 28*28 so the number of neurons will be 784. Output layer for MNIST consists of 10 neurons since the number of digits (so the number of classes for classification as well) equals to 10 (from 0 to 9). In hidden layers amount of neurons can be different.

Each neuron have activation function. The activation function is a formula which changes the input number. There are many different activation functions such as sigmoid, tanh, softmax, relu, etc. You can read more about them here: https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity

After we chose amount of layers and number of neurons for each layer, they will be automatically connected to each other (each neuron from a layer is connected to each neuron of the next layer). The stucture of a neural networks looks like this:

<img src="https://miro.medium.com/v2/resize:fit:1358/1*3fA77_mLNiJTSgZFhYnU0Q.png" width=550 height=350>

Each of this connection has *weight* which is a number that is usigng in the general formula for neural networks. This formula is:

<img src="https://machinelearningmastery.com/wp-content/uploads/2021/08/neural_networks_5.png" width=600 height=230>

Where x_i is *output* from each neuron from previous layer, w_i is *weight* of each connection, b is bias (which is just a constant value) and z is the number that goes to a neuron in next layer.


After creating a class with configuration of the network we have to choose loss function and the optimizer. They will be used in training and evaluation functions.

The loss function is a method of evaluating how well your machine learning algorithm models your featured data set. In other words, loss functions are a measurement of how good your model is in terms of predicting the expected outcome.

Optimizer ties together the loss function and model parameters by updating the model in response to the output of the loss function. In simpler terms, optimizers shape and mold your model into its most accurate possible form by futzing with the weights. The loss function is the guide to the terrain, telling the optimizer when it’s moving in the right or wrong direction.

The training function is the overall algorithm that is used to train the neural network to recognize a certain input and map it to an output. Inside the function you are computing loss function and depending on it update weights.

The evaluation function is used to evaluate the performance of the trained model. In other words, the evaluation function is used to measure how well your model is performing on new data.


Now we can move to the coding part. First we import all needed libraries including ptseries and PyTorch:

In [ ]:
# First, perform the relevant imports and navigate to the root folder
import os
import torch
import torch.nn as nn

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

from ptseries.models.utils import calculate_n_params, calculate_output_dim
from ptseries.models.pt_layer import PTLayer
from ptseries.algorithms.classifiers import VariationalClassifier
from ptseries.algorithms.classifiers.utils import create_dataloader

***PTLayer*** is a quantum layer for a neural network. It was built to work with PyTorch so it will be easy to integrate it to classical neural networks.

We will build a hybrid neural network for binary classification. So first of all we will create a dataset for our classifier.

In [ ]:
data = 4*torch.rand((200,2))-2
labels = torch.where(data[:,0]**2+data[:,1]**2 < 2, 1, 0).unsqueeze(1).to(torch.float32)
plt.scatter(data[:,0], data[:,1], c=labels)
plt.axis('equal')
plt.show()

On the plot you can see the training data. So the next step we will set a train dataloader and its batch size:

In [ ]:
train_dataloader = create_dataloader(data, labels, batch_size=16)

Next step is to create a hybrid neural network using PyTorch and PTLayer.

### Task 1: Set the size of inputs

The size of input size is determined by the dataset object size. In the code below you should set the parameter "self.input_size" as the size of the dataset object. 

### Task 2: Play with the neural network structure

You can modify the neural network structure by adding more layers, changing the number of neurons, using different activation functions, etc. The current structure is described in "self.net" parameter in the code below.

In [ ]:
class Model(nn.Module):

    def __init__(self, tbi_params=None):
        
        super().__init__()
        
        self.input_size = ...
        self.input_state = (0, 1, 0)
        self.tbi_params = tbi_params
        self.observable = 'avg-photons'

        # we use calculate_n_params to determine the number of beam splitter angles
        n_params = calculate_n_params(self.input_state, tbi_params=self.tbi_params)
        n_outputs = calculate_output_dim(
            self.input_state,
            tbi_params=self.tbi_params,
            observable=self.observable
            )
        
        self.net = nn.Sequential(
            nn.Linear(self.input_size, 100),
            nn.ReLU(),
            nn.Linear(100, n_params),
            PTLayer(
                self.input_state,
                in_features=n_params,
                observable=self.observable,
                tbi_params=self.tbi_params,
                n_samples=200
            ),
            nn.Linear(n_outputs, 100),
            nn.ReLU(),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

Time for training! Before training we will print model to make sure everything is right:

In [ ]:
model = Model()
print(model)


Defining loss function. You can choose your own loss function which can give better performance.

In [ ]:
loss_function = nn.BCELoss()
classifier = VariationalClassifier(model, loss_function)

### Task 3: Set up training parameters
To make training of the neural network. You should set up your learning rate and number of epochs.

In [ ]:
classifier.train(
    train_dataloader,        # pytorch dataloader containing the training data
    learning_rate=...,       # very small number 
    epochs=...,              # number of training iterations
    print_frequency=5,
    verbose=True
)

Evaluating the results of the training. We can compare graphs of the training data and the results:

In [ ]:
# Create some new test data and perform inference
data_test = 4*torch.rand((200,2))-2
predictions = classifier.forward(data_test)

# Convert predictions to blue/yellow binary labels and plot the result
binarized_predictions = torch.where(predictions < 0.5, 0, 1).unsqueeze(1)
plt.scatter(data_test[:,0], data_test[:,1], c=binarized_predictions)
plt.axis('equal')
plt.show()

Also we will check the accuracy of the NN:

In [ ]:
true_labels = []
for _, labels in train_dataloader:
    true_labels.extend(labels.numpy())

# Convert true labels to a tensor
true_labels = torch.tensor(true_labels)

# Assuming binary classification, round predictions to 0 or 1
predicted_labels = torch.round(predictions)

# Calculate accuracy
accuracy = torch.mean((predicted_labels == true_labels).float())

print(f'Accuracy: {accuracy.item() * 100:.2f}%')

#### Congrats! Now you know what are hybrid neural networks and how to create and train them! So we can move to the real hardware.

## 4. Real Quantum Hardware Use

Now that you have trained your hybrid quantum-classical neural network, you can use it to make predictions on real quantum hardware.

Here is an example of how to use the real Orca computer:

A connection to the device must be opened. We can do this by calling the `create_tbi` function with the `tbi_type` set to PT-1. Note that the object has some differences to the simulated backends you may have used previously. Specifically, you cannot specify parameters such as beam splitter loss becasue this is now a property of the device. For a full comparison, please consult the SDK documentation.

In [ ]:
time_bin_interferometer = create_tbi(
    tbi_type="PT-1",
    ip_address="192.168.34.2")

This object has methods that allow us to check the connection between the computer and the PT-1. We can also check the status of the hardware.

### Task 1: Solving MaxCut using BBS on real Orca computer

Solve MaxCut problem using BBS. Change tbi_params in the BBS to use the real Orca device.

***Important! The maximum numer of parameters (thetas) that is availible is 5 so make the instances of MaxCut with maximum 6 vertices***

For example, suppose that we have a set of different computers, each with different types of connections. Some computers have bluetooth, some have USB ports, HDMI ports, etc. We want to split our set of computers into two groups for two different projects, but it's very important that the two groups can connect to each other. The problem is sometimes the wires and connections don't work! How can we be sure that we have the best chance at remaining connected?

One way to solve this problem is with the maximum cut problem. If we think of our set of computers as a graph (a node/vertex for each computer), and draw an edge between computers that can connect to each other, we have a model of our network. If we look for a maximum cut in our graph, then we are looking for a way to split the nodes into two groups so that there are as many edges as possible between the groups. In our computer set, this means that we have two groups with as many connections as possible between the two groups. Now if one connection goes down, we have many more to use! This way we have created a more resilient network by providing many redundant connections between groups in case one connection fails.

Below we see a simple network of five nodes and three different ways to split the set of nodes into two groups. The dashed edges in each graph highlight the cut edges.

<img src="https://github.com/dwave-examples/maximum-cut/raw/master/readme_imgs/cut_examples.png">

Then we have to transform this problem to QUBO matrix so the BBS can solve it. The objective function that we are looking to optimize is maximizing the number of cut edges. To count how many cut edges we have given a partition of the nodes (assignment of our binary variables), we consider a single edge in a graph in the table below. We only want to count an edge if the endpoints are in different subsets, and so we assign a 1 for the edge_score column in this case and a 0 otherwise.
| $x_i$ | $x_j$ | edge\_score $(i,j)$ |
|:-----:|:-----:|:-------------------:|
|   0   |   0   |          0           |
|   0   |   1   |          1           |
|   1   |   0   |          1           |
|   1   |   1   |          0           |

From this table, we see that we can use the expression x_i+x_j-2x_ix_j to calculate the edge_score column in our table. Now for our entire graph, our objective function can be written as shown below, where the sum is over all edges in the graph.

<img src="https://github.com/dwave-examples/maximum-cut/raw/master/readme_imgs/QUBO.png">

Since our system is used to minimize an objective function, we must convert this maximization problem to a minimization problem by multiplying the expression by -1. Our final QUBO expression is the following.

<img src="https://github.com/dwave-examples/maximum-cut/raw/master/readme_imgs/final_QUBO.png">

Now we can use the BBS to solve our maximum cut problem. First step is to create an instance of MaxCut, then convert it to QUBO and solve using BBS. Then you can try to use the actual hardware to solve the instance.

In [ ]:
# You can copy some code here and modify it here.

### Task 2: Hybrid neural network with the real Orca computer

Change tbi_params in the hybrid neural network above so that it will train with the real Orca device. To make the task more complex you can change number of loops in the tbi_params but then also number of thetas will increase which means it must be calculated inside the HNN.

***Important! The maximum numer of parameters (thetas) that is availible is 5 so make the input objects with maximum size equal to 6***

In [ ]:
# You can copy neural network here or create a new one with and change tbi_params in it.

#### Congrats! You finished the tutorial! Now you know how to use real quantum device and apply it to neural networks.